In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!ls "/content/drive/My Drive/Colab Notebooks"

'Copy of Assignment6.ipynb'	    numpy.ipynb
'Copy of Week-1 Assignment.ipynb'   Pandas.ipynb
'Copy of Week-2 Assignment.ipynb'   simplilearn.ipynb
'Copy of Week-3 Assignment.ipynb'  'Summer internship TASK-2.ipynb'
'Copy of Week-4 Assignment.ipynb'   SUMMER_INTERNSHIP_TASK2.ipynb
'Copy of Week-5 Assignment.ipynb'  'SummerInternshipTask(NLP)2.ipynb'
'Copy of Week-7 Assignment.ipynb'  'Summer_Intership_Task1 (1).ipynb'
'Copy of Week-8 Assignment.ipynb'   Summer_Intership_Task1.ipynb
 DM.ipynb			    Untitled0.ipynb
 en-te_parallel_corpus.csv	    Untitled1.ipynb
 Matplotlib.ipynb


In [6]:
project_path="/content/drive/My Drive/Machine_Translation"

In [7]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

In [10]:
import pandas as pd
df=pd.read_csv("en-te_parallel_corpus.csv")
english_sentences=df['en_sent']
telugu_sentences=df['te_sent']


In [11]:
data = pd.DataFrame({"english_sentences":english_sentences,"telugu_sentences":telugu_sentences})
data

,english_sentences,telugu_sentences
0,Three members of a family were killed in the i...,ఓకే కుటుంబానికి చెందిన ముగ్గురు మృతిచెందటంతో ఈ...
1,"In Monaco-Ville, street signs are printed in b...","మొనాకో-విల్లెలో, ఫ్రెంచ్, మోనెగస్క్యూ రెండింటి..."
2,The same has been confirmed in a report carrie...,ప్రముఖ బీబీసీ నివేదిక ఇదే విషయం పేర్కొంది.
3,In this state they can easily survive one year...,సంవత్సరానికి అదనంగా ఒక రోజునో లేక ఒక నెలనో చేర...
4,Annual International Yoga Day is a modern cele...,ఇలా అంతర్జాతీయ యోగ దినోత్సవం భారతీయ సంస్కార ప్...
...,...,...
29995,The film has Nayanathara as the primary female...,ఆ సినిమాలో మొదట నయనతారను హీరోయిన్‌గా ఎంపిక చేయ...
29996,He carries out this task according to his own ...,"ఆ ప్రతీ సభ్యుడు తన సొంత సామర్థ్యాన్ని బట్టి, త..."
29997,The Muslims can have four wives at a time.,తమ మతాచారం ప్రకారం ముస్లింలు ఏకకాలంలో నలుగురు ...
29998,After Karnataka High Court's disappointing jud...,"కాగా, జయలలిత బెయిల్ పిటిషన్‌పై కర్ణాటక హైకోర్ట..."


In [12]:
# clean english sentances
def clean_eng(text):
    # Lowercase all characters
    text = text.lower()
    # Remove quotes
    text = re.sub("'", '', text)
    # Remove all the special characters
    exclude = set(string.punctuation) # Set of all special characters
    text = ''.join([c for c in text if c not in exclude])
    # Remove all numbers from text
    remove_digits = str.maketrans('', '', digits)
    text = text.translate(remove_digits)
    # Remove extra spaces
    text= text.strip()

    return text

In [13]:
# clean telugu sentances
def clean_tel(text):
    # Lowercase all characters
    text = text.lower()
    # Remove quotes
    text = re.sub("'", '', text)
    # Remove all the special characters
    exclude = set(string.punctuation) # Set of all special characters
    text = ''.join([c for c in text if c not in exclude])
    # Remove all numbers from text
    remove_digits = str.maketrans('', '', digits)
    text = text.translate(remove_digits)
    # Remove Telugu numbers from text
    text = re.sub("[౦౧౨౩౪౫౬౭౮౯]", '', text)
    # Remove extra spaces
    text= text.strip()
    text = '<s>'+ text + '</s>'
    return text

In [14]:
# clean text
data_df = data.copy()
data_df["english_sentences"] = data_df["english_sentences"] .apply(lambda x: clean_eng(x))
data_df["telugu_sentences"] = data_df["telugu_sentences"] .apply(lambda x: clean_tel(x))

In [15]:
# Vocabulary of English
all_eng_words=set()
for eng in data_df.english_sentences:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

In [16]:
# Vocabulary of French 
all_telugu_words=set()
for tel in data_df.telugu_sentences:
    for word in tel.split():
        if word not in all_telugu_words:
            all_telugu_words.add(word)

In [17]:
# Max Length of source sequence
lenght_list=[]
for l in data_df.english_sentences:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

20

In [18]:
# Max Length of target sequence
lenght_list=[]
for l in data_df.telugu_sentences:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

184

In [19]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_telugu_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_telugu_words)
num_encoder_tokens, num_decoder_tokens

(24819, 68082)

In [20]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

68083

In [21]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [22]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [23]:
data_df = shuffle(data_df)
data_df.head(10)

,english_sentences,telugu_sentences
28779,sejal sharma has committed suicide and tv indu...,<s>ప్రముఖ హిందీ బుల్లితెర నటి సెజల్ శర్మ ఆత్మహ...
26830,the other five have been identified and would ...,<s>మరో ఐదుగురు పరారీలో ఉన్నారని వారిని కూడా త్...
714,indian security forces gave a befitting reply ...,<s>భారత భద్రతా అధికారులు కూడా పాక్‌ బృందాల కాల...
18561,the devotees believe that the nandi idol in fr...,<s>ఈ ఆలయంలో ఉన్న పెద్ద నందీశ్వరుడి విగ్రహం అంత...
10042,bumrah is an absolutely quality bowler,<s>బుమ్రా అద్భుతమైన బౌలర్‌</s>
12928,i have come here to offer my respect to venera...,<s>నేను ఇక్కడకు వచ్చింది కఠోర శ్రమజీవి అయిన గౌ...
7714,india thus took a lead in the fivematch series,<s>దీంతో ఐదు మ్యాచ్‌ల సిరీస్‌లో భారత్ ఆధిక్యం...
5453,bihar chief minister nitish kumar and rjd pres...,<s>ఆయన ఇక్కడ ముఖ్యమంత్రి నితీష్‌కుమార్‌ను ఆర్‌...
19951,the seats where the elections will be held ar...,<s>దక్షిణ పరగణాలు దక్షిణ కోలకతా హుగ్లి జిల్లా...
15682,the watch pairs with an iphone or iphone s,<s>ఐఫోన్ ఇంకా ఐఫోన్ ఎస్ స్మార్ట్‌ఫోన్‌లను ఈ వా...


In [24]:
# Train - Test Split
X, y = data_df.english_sentences, data_df.telugu_sentences
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((27000,), (3000,))

In [25]:
X_train.to_pickle(project_path+'X_train.pkl')
X_test.to_pickle(project_path+'X_test.pkl')

In [26]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

#Encoder - Decoder Model Architecture

In [27]:
latent_dim = 50

In [28]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [29]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [30]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [31]:
batch_size = 128
epochs = 30
train_samples_steps = len(X_train) // batch_size
val_samples_steps = len(X_test) // batch_size

In [32]:
# generate train and test datra
train_gen = generate_batch(X_train, y_train, batch_size = batch_size)
test_gen = generate_batch(X_test, y_test, batch_size = batch_size)

In [33]:
# Defining a helper function to save the model after each epoch 
# in which the loss decreases 
filepath = project_path+'NMT_model_enc_dec.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
# Defining a helper function to reduce the learning rate each time 
# the learning plateaus 
reduce_alpha = ReduceLROnPlateau(monitor ='val_loss', factor = 0.2,patience = 1, min_lr = 0.001)
# stop traning if there increase in loss
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)
callbacks = [checkpoint, es, reduce_alpha] 

In [ ]:
# train the model
model.fit_generator(generator = train_gen,steps_per_epoch = train_samples_steps,epochs=epochs,validation_data = test_gen,validation_steps = val_samples_steps,callbacks = callbacks)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


# Inference Setup

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Decode Sample Sequences

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

# Evaluation on Train Dataset

In [ ]:
test_gen = generate_batch(X_test, y_test, batch_size = 1)
for k in range(10):
    (input_seq, actual_output), _ = next(test_gen)
    decoded_sentence = decode_sequence(input_seq)
    print('Input English sentence:', X_test[k:k+1].values[0])
    print('Actual Telugu Translation:', y_test[k:k+1].values[0][6:-4])
    print('Predicted Telugu Translation:', decoded_sentence[:-4])
    print("\n")